In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import warnings
import os.path
import info
#import lightgbm as lgb
from utils import *

warnings.filterwarnings("ignore")

today = datetime.today()
print("Today's date:", today.strftime("%Y-%m-%d"))

path_pc = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/'
path_img = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/PnL images/'+today.strftime("%Y%m%d")[-6:]+'/'

count = 0
day = today
while True:
    path_scoreboard = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/Score Boards individual symbol/'+day.strftime("%Y%m%d")[-6:]+'/'
    if os.path.exists(path_scoreboard+'3988.HK_scoreboard_knn.csv') and len(os.listdir(path_scoreboard)) > 0:
        print('\nLatest scoreboard folder: \n', path_scoreboard)
        break
    else:
        day = day - to_days(1)
        count += 1
        if count > 7:
            print("No valid scoreboard path. Please check.")
            break

Today's date: 2020-03-25

Latest scoreboard folder: 
 C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/Score Boards individual symbol/200320/


In [2]:
#path_scoreboard = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/Score Boards individual symbol/200304/'

In [3]:
outcomes_new = load_latest(today, 'outcomes_', path_pc)

Trying  outcomes_2020-03-25.csv
Trying  outcomes_2020-03-24.csv
Loading file:  outcomes_2020-03-24.csv


In [4]:
# outcomes_new = outcomes_new.drop(['log_return_10', 'return_10', 'log_return_5', 'return_5'], axis=1)
outcomes_new = outcomes_new.drop(['log_return_10', 'return_10'#,
                                  #'log_return_5', 'return_5',
                                  #'log_return_4', 'return_4',
                                  #'log_return_3', 'return_3',
                                  #'log_return_2', 'return_2'
                                 ], axis=1)

In [5]:
features_selected = ['(close/ema)-1',
 '(close/sma)-1',
 '1-lower/close',
 #'bbands50_lower',
 'bbands50_lower_slope',
 #'bbands50_upper',
 'bbands50_upper_slope',
 #'close',
 'close_pct_change_5_day',
 'close_scaled50',
 'close_slope',
 #'ema50',
 'ema50_scaled50',
 'ema50_slope',
 #'high',
 'high_scaled50',
 'high_slope',
 'intraday_chg',
 'log volume',
 #'low',
 'low_scaled50',
 'low_slope',
 #'macd',
 #'macd_signal',
 'macd_signal_pct_diff',
 'momentum',
 #'open',
 'open_scaled50',
 'open_slope',
 'rsi14',
 'rsi14_slope',
 #'sma50',
 'sma50_scaled50',
 'sma50_slope',
 'upper/close-1',
 'volatility50',
 'volatility50_ratio',
 #'volume',
 'volume_pct_change_1_day',
 'volume_scaled50',
 'volume_slope',
 'past_return_1',
 'past_return_2',
 'past_return_3',
 'past_return_4',
 'past_return_5',
 'past_return_10',
 'past_log_return_1',
 'past_log_return_2',
 'past_log_return_3',
 'past_log_return_4',
 'past_log_return_5',
 'past_log_return_10'
 ]

In [6]:
last_date = outcomes_new.index.get_level_values('date')[-1].to_pydatetime()
print ("Last date in dataset: ", last_date.strftime("%Y-%m-%d"))

Last date in dataset:  2020-03-24


In [7]:
#master_scoreboard = load_latest(last_date, 'master_scoreboard_', path_pc)

In [8]:
#scoreboard_last_date = 

In [9]:
outcomes_new_dropna = outcomes_new.dropna()

In [10]:
prob_threshold = 0.7
test_period = 100
test_size = 1
valid_size = 400
training_size = 2000
valid_test_gap = 4
q_upper = 0.9
q_lower = 0.1
# return_col = 'log_return_1'
# return_col_actual = 'return_1'
#return_col = 'log_return_2'
#return_col_actual = 'return_2'
# return_col = 'log_return_3'
# return_col_actual = 'return_3'
return_col = 'log_return_5'
return_col_actual = 'return_5'
# return_col = 'log_return_10'
# return_col_actual = 'return_10'
max_depth_range = np.arange(2,5,1)
num_leaves_range = np.arange(9,15,1)
p_range = [1,2]
leaf_size_range= [1] #list(range(1,50))
n_neighbors_range = list(range(1,30))

## Back Test each symbol one by one, store results in master scoreboard

In [11]:
from pathlib import Path

#Create new folder for scoreboards
path_scoreboard_new = 'C:/Users/admin/Desktop/AI Plan/Finance with AI/Notebooks/Score Boards individual symbol/'+today.strftime("%Y%m%d")[-6:]+'/'
print("Creating new folder: ", today.strftime("%Y%m%d")[-6:]+'/')
Path(path_scoreboard_new).mkdir(parents=True, exist_ok=True)

Creating new folder:  200325/


In [12]:
master_scoreboard = pd.DataFrame()

for symbol_loop in list(info.multiplier.keys()):
    
    #print("-"*80)
    #print("\nWorking on ", symbol_loop)
    
    outcomes_new_dropna = outcomes_new.dropna()
    outcomes_new_dropna = outcomes_new_dropna.loc[outcomes_new_dropna.index.get_level_values('symbol')==symbol_loop]
    
    #load latest scoreboard for this symbol
    score_board_filename = symbol_loop + '_scoreboard_knn.csv'
    if os.path.exists(path_scoreboard + score_board_filename):
        score_board = pd.read_csv(path_scoreboard + score_board_filename)
    else:
        print("Skipping ", symbol_loop, " as previous data doesn't exist.")
        continue
    
    #get all trading dates from dataset of this symbol
    data_dates = list(set(outcomes_new_dropna.index.get_level_values(0)))
    data_dates = sorted(data_dates)
    data_converted_dates = []
    for ts in data_dates:
        data_converted_dates.append(ts.to_pydatetime())
    
    #calculate training start dates according to test periods, train, valid and test sizes
    total_num_days = len(data_converted_dates)
    num_periods = int((total_num_days -(valid_size + training_size + valid_test_gap))/test_size)
    start_dates = []
    for i in range(1, num_periods):
        start_dates.append(data_converted_dates[(test_size*-i) - valid_test_gap - valid_size - training_size])
    start_dates = sorted(start_dates)

    #skip symbols that are relatively new and has not enough trading days for training 
    if len(start_dates) < test_period:
        print("Skipped ", symbol_loop, " due to insuffient trading days.")
        continue
    
    #Calculate dates for new test data that are not in scoreboard
    score_board_dates = []
    for date_str in score_board['01) start_date']:
        score_board_dates.append(datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S'))
    new_start_dates = list(set(start_dates[-test_period:]) - set(score_board_dates))
    new_start_dates = sorted(new_start_dates)

    #Calculate next day open
    next_day_open = lambda x: x.shift(-1)
    outcomes_new_dropna['next_day_open'] = outcomes_new_dropna.groupby(level='symbol').apply(next_day_open)['open']
    
    if new_start_dates == []:
        print("No new data are found for {}. No backtest is done.".format(symbol_loop))
    else:
        print("Training for ", symbol_loop)

        for start_date in new_start_dates:
            counter = data_converted_dates.index(start_date)
            start_date_valid = data_converted_dates[counter + training_size -1]
            end_date_valid = data_converted_dates[counter + training_size + valid_size -1]
            start_date_test = data_converted_dates[counter + training_size + valid_test_gap + valid_size-1]
            end_date_test = data_converted_dates[counter + training_size + valid_test_gap + valid_size + test_size-1]
            #print("start training date: ", start_date)
            #print("Valid dataset period: ", start_date_valid, " - ", end_date_valid)
            #print("Test datasset period: ", start_date_test, " - ", end_date_test)

            #split into train, valid and test sets
            X_y_train, X_y_valid, X_y_test = train_valid_test_split(outcomes_new_dropna, start_date,
                                                                    start_date_valid, end_date_valid, start_date_test, end_date_test)

            #calculate upper threshold in training set, then create targets for both valid and test
            X_y_train, X_y_valid, X_y_test = add_target_upper(X_y_train, X_y_valid,
                                                              X_y_test, q_upper, 'target', return_col)

            X_y_test['symbol'] = X_y_test.index.get_level_values('symbol')

            #downsample the training set's negative data points
            X_y_train_resampled = downsample(X_y_train, 'target', test_ratio=0.11, random_seed=11)

            #create 2 extra sets for calculating gain
            X_valid_close = X_y_valid[['close',return_col_actual]]
            X_test_close = X_y_test[['close',return_col_actual, 'symbol','next_day_open']]

            num_shares = []

            for i, row in X_test_close.iterrows():
                num_shares.append(info.board_lots[row[2]] * info.multiplier[row[2]])

            X_test_close['num_shares'] = num_shares

            #split into features and target sets
            X_train, y_train = feature_target_split(X_y_train_resampled, features_selected, 'target')
            X_valid, y_valid = feature_target_split(X_y_valid, features_selected, 'target')
            X_test, y_test = feature_target_split(X_y_test, features_selected, 'target')
            
            (best_model, best_auc_model, max_auc, p_at_max_auc, leaf_size_at_max_auc,
             n_neighbors_at_max_auc, p_at_max_tt, leaf_size_at_max_tt, n_neighbors_at_max_tt,
             y_class_pred) = knn_train(X_train, y_train, X_valid, y_valid, X_valid_close, p_range,
                                       leaf_size_range, n_neighbors_range, return_col_actual, prob_threshold = 0.7, sign = 1)

            #calculate test gain using best gain model and best pres model
            test_total_gain, total_amount_spent, y_class_pred = total_actual_gain_knn(best_model, X_test, X_test_close,
                                                 y_test, prob_threshold, return_col_actual)
            test_auc_total_gain, total_auc_amount_spent, y_class_auc_pred = total_actual_gain_knn(best_auc_model, X_test, X_test_close,
                                                     y_test, prob_threshold, return_col_actual)
            
            score_board = score_board.append({'01) start_date': start_date, '02) start_date_valid':start_date_valid, '03) start_date_test': start_date_test,
                        '04a) test_total_gain': test_total_gain,
                        '04b) total_amount_spent': total_amount_spent,
                       '07) max_auc': max_auc,
                       '08) test_auc_total_gain': test_auc_total_gain,        
                        #'09) optimal_precision_depth': optimal_precision_depth,
                        #'10) optimal_precision_num_leaves': optimal_precision_num_leaves,
                        '10a) total_auc_amount_spent': total_auc_amount_spent}, ignore_index=True)

        gain_loss_ratio = gain_vs_loss(score_board['04a) test_total_gain'])
        gain_loss_ratio_auc = gain_vs_loss(score_board['08) test_auc_total_gain'])
        ######################################################################################
        #   MAY NEED TO DROP FIRST FEW ROWS IN SCORE_BOARD TO MAINTAIN EXACTLY 100 ROWS      #
        ######################################################################################
        score_filename = symbol_loop + '_scoreboard_knn.csv'
        print("Saving score board for ", symbol_loop)
        save_csv(score_board, path_scoreboard_new, score_filename)
        
        master_scoreboard = master_scoreboard.append({'symbol': symbol_loop,
                                                  'Test Total Gain': score_board['04a) test_total_gain'].sum(),
                                                  'Gain Loss Ratio' : gain_loss_ratio,
                                                  'Gain Loss Ratio AUC' : gain_loss_ratio_auc,
                                                  'Test AUC Total Gain': score_board['08) test_auc_total_gain'].sum(),
                                                  'Profit Percentage': score_board['04a) test_total_gain'].sum()/(score_board['04b) total_amount_spent'].mean()*5),
                                                  'Profit Percentage Max AUC': score_board['08) test_auc_total_gain'].sum()/(score_board['10a) total_auc_amount_spent'].mean()*5),
                                                  'Total amount spent': score_board['04b) total_amount_spent'].mean(),
                                                  'Total AUC amount spent': score_board['10a) total_auc_amount_spent'].mean()}, ignore_index=True)

        print("Saving plots for ", symbol_loop)
        plot_filename_total = symbol_loop + '_knn_total.png'
        plot_filename_pres = symbol_loop + '_knn_auc.png'

        fig_total = score_board['04a) test_total_gain'].cumsum().plot().get_figure()
        fig_total.savefig(path_img+plot_filename_total)

        fig_pres = score_board['08) test_auc_total_gain'].cumsum().plot().get_figure()
        fig_pres.savefig(path_img+plot_filename_pres)
        fig_pres.clf()
        fig_total.clf()

Training for  0001.HK
Saving score board for  0001.HK
Saving plots for  0001.HK
Training for  0002.HK
Saving score board for  0002.HK
Saving plots for  0002.HK
Training for  0003.HK
Saving score board for  0003.HK
Saving plots for  0003.HK
Training for  0005.HK
Saving score board for  0005.HK
Saving plots for  0005.HK
Training for  0006.HK
Saving score board for  0006.HK
Saving plots for  0006.HK
Training for  0011.HK
Saving score board for  0011.HK
Saving plots for  0011.HK
Training for  0012.HK
Saving score board for  0012.HK
Saving plots for  0012.HK
Training for  0016.HK
Saving score board for  0016.HK
Saving plots for  0016.HK
Training for  0017.HK
Saving score board for  0017.HK
Saving plots for  0017.HK
Training for  0019.HK
Saving score board for  0019.HK
Saving plots for  0019.HK
Training for  0027.HK
Saving score board for  0027.HK
Saving plots for  0027.HK
Training for  0066.HK
Saving score board for  0066.HK
Saving plots for  0066.HK
Training for  0083.HK
Saving score board

In [13]:
master_scoreboard.sort_values(by=['Gain Loss Ratio'], ascending=False)[0:5]

,Gain Loss Ratio,Gain Loss Ratio AUC,Profit Percentage,Profit Percentage Max AUC,Test AUC Total Gain,Test Total Gain,Total AUC amount spent,Total amount spent,symbol
26,1.637631,2.371179,0.157196,0.270445,15700.0,9150.0,11610.472973,11641.554054,0939.HK
33,1.397290,2.136719,0.270490,0.554131,26190.0,16710.0,9452.635135,12355.337838,1177.HK
41,1.365145,0.686869,0.119777,-0.150125,-6200.0,4400.0,8259.797297,7346.959459,2388.HK
21,1.283109,1.021545,0.119827,0.010006,580.0,7140.0,11592.837838,11917.162162,0700.HK
31,1.222906,1.528683,0.150065,0.294992,29860.0,15220.0,20244.594595,20284.594595,1093.HK


In [14]:
master_scoreboard.loc[master_scoreboard['Gain Loss Ratio'] == float("-inf")]

,Gain Loss Ratio,Gain Loss Ratio AUC,Profit Percentage,Profit Percentage Max AUC,Test AUC Total Gain,Test Total Gain,Total AUC amount spent,Total amount spent,symbol


In [15]:
if new_start_dates == []:
    print('No master scoreboard is saved.')
else:
    save_csv(master_scoreboard, path_pc, 'master_scoreboard_knn_'+last_date.strftime("%Y-%m-%d")+'.csv')
    print('Master scoreboard is saved to ', 'master_scoreboard_knn_'+last_date.strftime("%Y-%m-%d")+'.csv')

Master scoreboard is saved to  master_scoreboard_knn_2020-03-24.csv


# Read Master Scoreboard

In [16]:
master_scoreboard = pd.read_csv(path_pc+'master_scoreboard_knn_'+last_date.strftime("%Y-%m-%d")+'.csv')

In [17]:
master_scoreboard.sort_values(by=['Test Total Gain'], ascending=False)[0:5]

,Unnamed: 0,Gain Loss Ratio,Gain Loss Ratio AUC,Profit Percentage,Profit Percentage Max AUC,Test AUC Total Gain,Test Total Gain,Total AUC amount spent,Total amount spent,symbol
33,33,1.397290,2.136719,0.270490,0.554131,26190.0,16710.0,9452.635135,12355.337838,1177.HK
31,31,1.222906,1.528683,0.150065,0.294992,29860.0,15220.0,20244.594595,20284.594595,1093.HK
26,26,1.637631,2.371179,0.157196,0.270445,15700.0,9150.0,11610.472973,11641.554054,0939.HK
21,21,1.283109,1.021545,0.119827,0.010006,580.0,7140.0,11592.837838,11917.162162,0700.HK
41,41,1.365145,0.686869,0.119777,-0.150125,-6200.0,4400.0,8259.797297,7346.959459,2388.HK


In [18]:
master_scoreboard.sort_values(by=['Profit Percentage'], ascending=False)[0:5]

,Unnamed: 0,Gain Loss Ratio,Gain Loss Ratio AUC,Profit Percentage,Profit Percentage Max AUC,Test AUC Total Gain,Test Total Gain,Total AUC amount spent,Total amount spent,symbol
33,33,1.397290,2.136719,0.270490,0.554131,26190.0,16710.0,9452.635135,12355.337838,1177.HK
26,26,1.637631,2.371179,0.157196,0.270445,15700.0,9150.0,11610.472973,11641.554054,0939.HK
31,31,1.222906,1.528683,0.150065,0.294992,29860.0,15220.0,20244.594595,20284.594595,1093.HK
21,21,1.283109,1.021545,0.119827,0.010006,580.0,7140.0,11592.837838,11917.162162,0700.HK
41,41,1.365145,0.686869,0.119777,-0.150125,-6200.0,4400.0,8259.797297,7346.959459,2388.HK


In [19]:
master_scoreboard.sort_values(by=['Gain Loss Ratio AUC'], ascending=False)[0:5]

,Unnamed: 0,Gain Loss Ratio,Gain Loss Ratio AUC,Profit Percentage,Profit Percentage Max AUC,Test AUC Total Gain,Test Total Gain,Total AUC amount spent,Total amount spent,symbol
26,26,1.637631,2.371179,0.157196,0.270445,15700.0,9150.0,11610.472973,11641.554054,0939.HK
33,33,1.397290,2.136719,0.270490,0.554131,26190.0,16710.0,9452.635135,12355.337838,1177.HK
31,31,1.222906,1.528683,0.150065,0.294992,29860.0,15220.0,20244.594595,20284.594595,1093.HK
29,29,0.737964,1.465741,-0.161324,0.240285,12575.0,-8300.0,10466.722973,10289.864865,1044.HK
34,34,0.858824,1.233161,-0.059195,0.078733,5400.0,-2880.0,13717.297297,9730.540541,1398.HK


In [20]:
master_scoreboard.loc[master_scoreboard['Gain Loss Ratio'] == float("-inf")]

,Unnamed: 0,Gain Loss Ratio,Gain Loss Ratio AUC,Profit Percentage,Profit Percentage Max AUC,Test AUC Total Gain,Test Total Gain,Total AUC amount spent,Total amount spent,symbol
